In [ ]:
! pip install pandas yfinance supabase

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.9/169.9 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1


# Company Info

## Scraping Data

In [ ]:
import yfinance as yf
import pandas as pd
import logging
import os
import json
from concurrent.futures import ThreadPoolExecutor
from typing import List, Dict, Optional

# Configure logging for information and error tracking
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class CompanyInfoFetcher:
    """
    Class for fetching company information from Yahoo Finance
    """

    def __init__(self, output_dir: str = "company_data"):
        """
        Initialize CompanyInfoFetcher with output directory.

        Args:
            output_dir (str): Directory to store CSV files.
        """
        self.output_dir = output_dir

        # Create output directory if it doesn't exist
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            logger.info(f"Output directory created: {output_dir}")

    def fetch_company_info(self, ticker: str) -> Dict:
        """
        Fetch company information for a given ticker.

        Args:
            ticker (str): Stock symbol (e.g., "BBCA.JK")

        Returns:
            Dict: Dictionary containing company information
        """
        try:
            logger.info(f"Fetching company info for {ticker}")
            stock = yf.Ticker(ticker)

            # Get company information
            info = stock.info

            # Extract ONLY the requested information
            company_data = {
                'ticker': ticker,
                'address1': info.get('address1', None),
                'sector': info.get('sector', None),
                'website': info.get('website', None),
                'phone': info.get('phone', None),
                'longname': info.get('longName', None),
                'longbusinesssummary': info.get('longBusinessSummary', None)
            }

            logger.info(f"Successfully fetched company info for {ticker}")
            return company_data

        except Exception as ex:
            logger.exception(f"Failed to fetch company info for {ticker}: {ex}")
            return {
                'ticker': ticker,
                'error': str(ex)
            }

    def save_data_to_csv(self, data: Dict, ticker: str) -> Optional[str]:
        """
        Save company data to CSV file.

        Args:
            data (Dict): Company information dictionary
            ticker (str): Stock symbol

        Returns:
            Optional[str]: Path to CSV file if successful, None if failed
        """
        if not data or (len(data) <= 2 and 'error' in data):
            logger.warning(f"No data to save for {ticker}")
            return None

        try:
            # Clean ticker for filename
            ticker_clean = ticker.replace('.', '_')
            filename = f"{ticker_clean}_company_info.csv"
            filepath = os.path.join(self.output_dir, filename)

            # Convert to DataFrame and save
            df = pd.DataFrame([data])
            df.to_csv(filepath, index=False)

            # Also save as JSON for easier debugging
            json_path = os.path.join(self.output_dir, f"{ticker_clean}_company_info.json")
            with open(json_path, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=4)

            logger.info(f"Data successfully saved: {filepath}")
            return filepath

        except Exception as ex:
            logger.exception(f"Failed to save data to file for {ticker}: {ex}")
            return None

    def fetch_and_save_for_ticker(self, ticker: str) -> Dict:
        """
        Fetch and save company info for one ticker.

        Args:
            ticker (str): Stock symbol

        Returns:
            Dict: Dictionary with results
        """
        data = self.fetch_company_info(ticker)
        file_path = self.save_data_to_csv(data, ticker)

        return {
            'ticker': ticker,
            'file_path': file_path,
            'success': file_path is not None
        }

    def fetch_multi_tickers(self, tickers: List[str], use_threads: bool = True) -> List[Dict]:
        """
        Fetch company information for multiple tickers.

        Args:
            tickers (List[str]): List of stock symbols
            use_threads (bool): Use multithreading for data fetching

        Returns:
            List[Dict]: List of results for each ticker
        """
        logger.info(f"Starting data fetching for {len(tickers)} tickers")
        results = []

        if use_threads:
            # Use ThreadPoolExecutor for parallel execution
            with ThreadPoolExecutor(max_workers=min(10, len(tickers))) as executor:
                futures = {
                    executor.submit(self.fetch_and_save_for_ticker, ticker): ticker
                    for ticker in tickers
                }

                for future in futures:
                    ticker = futures[future]
                    try:
                        results.append(future.result())
                    except Exception as ex:
                        logger.exception(f"Error in thread for ticker {ticker}: {ex}")
                        results.append({
                            'ticker': ticker,
                            'file_path': None,
                            'success': False,
                            'error': str(ex)
                        })
        else:
            # Sequential approach
            for ticker in tickers:
                results.append(self.fetch_and_save_for_ticker(ticker))

        logger.info(f"Data fetching completed")
        return results

    def generate_summary(self, results: List[Dict]) -> None:
        """
        Print a summary of data fetching results.

        Args:
            results (List[Dict]): Results from fetch_multi_tickers
        """
        success_count = sum(1 for r in results if r.get('success', False))
        failed_count = len(results) - success_count

        print("\n" + "="*80)
        print("COMPANY INFORMATION FETCHING SUMMARY")
        print("="*80)

        for result in results:
            status = "SUCCESS" if result.get('success', False) else "FAILED"
            print(f"{result['ticker']}: {status} {result.get('file_path', '')}")

        print("\n" + "="*80)
        print(f"Total: {len(results)} tickers")
        print(f"Success: {success_count}, Failed: {failed_count}")
        print("="*80 + "\n")

def main():
    # List of stocks to fetch data for
    tickers = ["PGEO.JK", "ANTM.JK", "PTBA.JK", "FCX", "TINS.JK",
               "BBRI.JK", "BMRI.JK", "BBNI.JK", "INCO.JK", "TLKM.JK"]

    # Directory to save results
    output_dir = "company_data"

    # Initialize fetcher
    fetcher = CompanyInfoFetcher(output_dir=output_dir)

    # Fetch and save data
    results = fetcher.fetch_multi_tickers(
        tickers=tickers,
        use_threads=True
    )

    # Display summary
    fetcher.generate_summary(results)

if __name__ == "__main__":
    main()


COMPANY INFORMATION FETCHING SUMMARY
PGEO.JK: SUCCESS company_data/PGEO_JK_company_info.csv
ANTM.JK: SUCCESS company_data/ANTM_JK_company_info.csv
PTBA.JK: SUCCESS company_data/PTBA_JK_company_info.csv
FCX: SUCCESS company_data/FCX_company_info.csv
TINS.JK: SUCCESS company_data/TINS_JK_company_info.csv
BBRI.JK: SUCCESS company_data/BBRI_JK_company_info.csv
BMRI.JK: SUCCESS company_data/BMRI_JK_company_info.csv
BBNI.JK: SUCCESS company_data/BBNI_JK_company_info.csv
INCO.JK: SUCCESS company_data/INCO_JK_company_info.csv
TLKM.JK: SUCCESS company_data/TLKM_JK_company_info.csv

Total: 10 tickers
Success: 10, Failed: 0



## Pre-processing Data

In [ ]:
import pandas as pd
import os
import glob

# Directory where CSV files are stored
data_dir = "company_data/"

# Find all CSV files in directory
csv_files = glob.glob(os.path.join(data_dir, "*_company_info.csv"))

# Loop through all CSV files
for file_path in csv_files:
    try:
        # Read CSV file
        df = pd.read_csv(file_path)

        # Extract ticker from filename
        file_name = os.path.basename(file_path)
        ticker_part = file_name.split('_company_info.csv')[0]

        # Reconstruct ticker if needed
        if 'JK' in ticker_part:
            ticker = ticker_part.replace('_', '.')
        else:
            ticker = ticker_part

        # Ensure ticker is in the data
        if 'ticker' not in df.columns or df['ticker'].iloc[0] != ticker:
            df['ticker'] = ticker

        # Clean up text fields - remove newlines and excessive spaces
        for col in ['longbusinesssummary', 'address1', 'longname']:
            if col in df.columns:
                df[col] = df[col].astype(str).str.replace('\n', ' ').str.replace('\r', ' ')
                df[col] = df[col].str.replace('nan', '').str.strip()
                df[col] = df[col].str.replace('\s+', ' ', regex=True)

        # Handle phone number formatting if needed
        if 'phone' in df.columns:
            df['phone'] = df['phone'].astype(str).str.replace('nan', '')

        # Handle website URL
        if 'website' in df.columns:
            df['website'] = df['website'].astype(str).str.replace('nan', '')

        # Handle sector
        if 'sector' in df.columns:
            df['sector'] = df['sector'].astype(str).str.replace('nan', '')

        # Ensure all expected columns exist
        required_columns = ['ticker', 'address1', 'sector', 'website', 'phone', 'longname', 'longbusinesssummary']
        for col in required_columns:
            if col not in df.columns:
                df[col] = ''

        # Keep only the required columns
        df = df[required_columns]

        # Save processed CSV
        df.to_csv(file_path, index=False)
        print(f"Preprocessing successful: {file_path}")
    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")

print("\nPreprocessing completed for all files!")

Preprocessing successful: company_data/BMRI_JK_company_info.csv
Preprocessing successful: company_data/PGEO_JK_company_info.csv
Preprocessing successful: company_data/BBNI_JK_company_info.csv
Preprocessing successful: company_data/PTBA_JK_company_info.csv
Preprocessing successful: company_data/BBRI_JK_company_info.csv
Preprocessing successful: company_data/INCO_JK_company_info.csv
Preprocessing successful: company_data/ANTM_JK_company_info.csv
Preprocessing successful: company_data/TINS_JK_company_info.csv
Preprocessing successful: company_data/FCX_company_info.csv
Preprocessing successful: company_data/TLKM_JK_company_info.csv

Preprocessing completed for all files!


## Push to Supabase

In [ ]:
import os
import glob
import json
from supabase import create_client, Client
import time
from google.colab import userdata

# Supabase credentials
supabase_url = userdata.get('SUPABASE_URL')
supabase_key = userdata.get('SUPABASE_KEY')

# Initialize Supabase client
supabase: Client = create_client(supabase_url, supabase_key)

print("Supabase client initialized successfully!")

def clean_record(record):
    """Clean a record to ensure it can be properly handled."""
    # Define the required fields
    required_fields = ['ticker', 'address1', 'sector', 'website', 'phone', 'longname', 'longbusinesssummary']

    clean_data = {}

    # Only keep the required fields
    for key in required_fields:
        value = record.get(key)

        # Handle NaN, None, and other problematic values
        if value is None or (isinstance(value, str) and value.lower() == 'nan'):
            clean_data[key] = None
        elif isinstance(value, float) and (value != value or value == float('inf') or value == float('-inf')):
            # Check for NaN or infinity
            clean_data[key] = None
        else:
            clean_data[key] = value

    return clean_data

def upload_to_supabase_from_json(json_file, table_name='company_info'):
    """Upload company data from JSON file to Supabase."""
    try:
        # Read JSON file
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # Clean the record and keep only required fields
        record = clean_record(data)
        ticker = record.get('ticker')

        if not ticker:
            print(f"No ticker found in {json_file}, skipping")
            return False

        # Check if ticker already exists
        response = supabase.table(table_name).select('*').eq('ticker', ticker).execute()

        if hasattr(response, 'data') and response.data and len(response.data) > 0:
            # Update existing record
            print(f"Updating record for {ticker}...")
            update_response = supabase.table(table_name).update(record).eq('ticker', ticker).execute()
            if hasattr(update_response, 'error') and update_response.error:
                print(f"Error updating record for {ticker}: {update_response.error}")
                return False
            else:
                print(f"✅ Updated record for {ticker}")
                return True
        else:
            # Insert new record
            print(f"Inserting new record for {ticker}...")
            insert_response = supabase.table(table_name).insert(record).execute()
            if hasattr(insert_response, 'error') and insert_response.error:
                print(f"Error inserting record for {ticker}: {insert_response.error}")
                return False
            else:
                print(f"✅ Inserted new record for {ticker}")
                return True

    except Exception as e:
        print(f"❌ Error processing {json_file}: {e}")
        return False

def upload_all_company_data_json(base_dir='company_data'):
    """Find and upload all company info JSON files."""
    total_records = 0
    json_files = glob.glob(f"{base_dir}/*_company_info.json")

    for json_file in json_files:
        filename = os.path.basename(json_file)
        print(f"\n🔄 Processing {filename}")

        if upload_to_supabase_from_json(json_file):
            total_records += 1

        # Small delay to prevent rate limiting
        time.sleep(0.2)

    print(f"\n✅ Total records uploaded to Supabase: {total_records}")
    return total_records

# Run the upload function using JSON files
result = upload_all_company_data_json()
print(result)

Supabase client initialized successfully!

🔄 Processing TLKM_JK_company_info.json
Updating record for TLKM.JK...
✅ Updated record for TLKM.JK

🔄 Processing PGEO_JK_company_info.json
Updating record for PGEO.JK...
✅ Updated record for PGEO.JK

🔄 Processing BMRI_JK_company_info.json
Updating record for BMRI.JK...
✅ Updated record for BMRI.JK

🔄 Processing INCO_JK_company_info.json
Updating record for INCO.JK...
✅ Updated record for INCO.JK

🔄 Processing PTBA_JK_company_info.json
Updating record for PTBA.JK...
✅ Updated record for PTBA.JK

🔄 Processing TINS_JK_company_info.json
Updating record for TINS.JK...
✅ Updated record for TINS.JK

🔄 Processing BBNI_JK_company_info.json
Updating record for BBNI.JK...
✅ Updated record for BBNI.JK

🔄 Processing BBRI_JK_company_info.json
Updating record for BBRI.JK...
✅ Updated record for BBRI.JK

🔄 Processing ANTM_JK_company_info.json
Updating record for ANTM.JK...
✅ Updated record for ANTM.JK

🔄 Processing FCX_company_info.json
Updating record for FC

# Price History

## Scraping Data

In [ ]:
import yfinance as yf
import pandas as pd
import logging
import os
from math import ceil
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
from typing import List, Dict, Tuple, Optional

# Konfigurasi logging untuk pencatatan informasi dan error
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class StockDataFetcher:
    """
    Kelas untuk mengambil data historis saham dari Yahoo Finance dengan dukungan
    untuk multi-timeframe dan multiple tickers.
    """
    # Dictionary untuk memetakan interval ke perkiraan jumlah bars per hari
    BARS_PER_DAY = {
        '1m': 390,  # 6.5 jam perdagangan × 60 menit
        '2m': 195,  # 6.5 jam perdagangan × 30 menit
        '5m': 78,   # 6.5 jam perdagangan × 12 menit
        '15m': 26,  # 6.5 jam perdagangan ÷ 15 menit
        '30m': 13,  # 6.5 jam perdagangan ÷ 30 menit
        '60m': 7,   # 6.5 jam perdagangan ÷ 60 menit
        '1h': 7,    # 6.5 jam perdagangan ÷ 1 jam
        '1d': 1     # 1 bar per hari
    }

    # Faktor koreksi berdasarkan data aktual vs teoretis
    CORRECTION_FACTORS = {
        '15m': 0.78,  # 33/42
        '30m': 0.88,  # 37/42
        '1h': 0.95,   # 40/42
        '1d': 1.0
    }

    # Estimasi jumlah maksimum baris per interval (berdasarkan data aktual)
    MAX_ROWS_ESTIMATION = {
        '15m': 856,  # Dari data pengguna
        '30m': 482,  # Dari data pengguna
        '1h': 283,   # Dari data pengguna
        '1d': 1000   # Tidak terbatas oleh 60 hari
    }

    # Interval intraday yang dibatasi oleh 60 hari
    INTRADAY_INTERVALS = ['1m', '2m', '5m', '15m', '30m', '60m', '1h']

    # Perkiraan jumlah hari trading dalam 60 hari kalender (disesuaikan dari data aktual)
    TRADING_DAYS_IN_60_CALENDAR_DAYS = 35  # Rata-rata dari 33, 37, dan 40

    def __init__(self, output_dir: str = "stock_data"):
        """
        Inisialisasi StockDataFetcher dengan direktori output.

        Args:
            output_dir (str): Direktori untuk menyimpan file CSV.
        """
        self.output_dir = output_dir

        # Buat direktori output jika belum ada
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            logger.info(f"Direktori output dibuat: {output_dir}")

    def get_max_rows_for_interval(self, interval: str) -> int:
        """
        Mengembalikan perkiraan jumlah maksimum baris yang dapat diambil untuk interval
        berdasarkan data aktual.

        Args:
            interval (str): Interval data (contoh: '15m')

        Returns:
            int: Jumlah maksimum baris data yang dapat diambil
        """
        if interval in self.MAX_ROWS_ESTIMATION:
            # Gunakan nilai dari data aktual jika tersedia
            return self.MAX_ROWS_ESTIMATION[interval]
        elif interval in self.INTRADAY_INTERVALS:
            # Untuk interval intraday lainnya, hitung dengan faktor koreksi
            avg_bars_per_day = self.BARS_PER_DAY.get(interval, 7)
            correction = self.CORRECTION_FACTORS.get(interval, 0.85)  # Default 0.85 jika tidak ada faktor khusus

            # Hitung jumlah maksimum baris dengan koreksi
            return int(avg_bars_per_day * self.TRADING_DAYS_IN_60_CALENDAR_DAYS * correction)
        else:
            # Untuk data non-intraday, tidak ada batasan khusus
            return float('inf')  # Tidak terbatas

    def calculate_period(self, row_limit: int, interval: str) -> Tuple[str, int]:
        """
        Menghitung periode pengambilan data berdasarkan row_limit dan interval.
        Untuk data intraday, yfinance hanya mendukung maksimal 60 hari.

        Args:
            row_limit (int): Jumlah baris data yang diinginkan.
            interval (str): Interval data (contoh: '15m').

        Returns:
            Tuple[str, int]:
            - Nilai period yang valid untuk yfinance
            - Jumlah baris yang sebenarnya akan diminta (disesuaikan jika melebihi batas)
        """
        # Untuk interval intraday, periksa batas maksimum
        if interval in self.INTRADAY_INTERVALS:
            max_rows = self.get_max_rows_for_interval(interval)

            # Jika row_limit melebihi maksimum yang mungkin dalam 60 hari
            if row_limit > max_rows:
                adjusted_row_limit = max_rows
                logger.info(f"Menyesuaikan row_limit untuk {interval} dari {row_limit} ke {adjusted_row_limit} (batas data aktual)")
            else:
                adjusted_row_limit = row_limit

            # Hitung jumlah hari yang diperlukan (tidak lebih dari 60)
            # Gunakan perkiraan hari trading berdasarkan data aktual
            if interval == '15m':
                estimated_days = min(60, ceil(adjusted_row_limit / self.BARS_PER_DAY[interval] * (1 / self.CORRECTION_FACTORS[interval])))
            elif interval == '30m':
                estimated_days = min(60, ceil(adjusted_row_limit / self.BARS_PER_DAY[interval] * (1 / self.CORRECTION_FACTORS[interval])))
            elif interval == '1h':
                estimated_days = min(60, ceil(adjusted_row_limit / self.BARS_PER_DAY[interval] * (1 / self.CORRECTION_FACTORS[interval])))
            else:
                avg_bars_per_day = self.BARS_PER_DAY.get(interval, 7)
                correction = self.CORRECTION_FACTORS.get(interval, 0.85)
                estimated_days = min(60, ceil(adjusted_row_limit / avg_bars_per_day * (1 / correction)))

            return f"{estimated_days}d", adjusted_row_limit
        else:
            # Untuk interval daily atau lebih besar
            days_needed = ceil(row_limit / self.BARS_PER_DAY.get(interval, 1))

            # Konversi ke periode yang sesuai
            if days_needed <= 60:
                return f"{days_needed}d", row_limit
            elif days_needed <= 730:  # ~2 tahun
                return f"{ceil(days_needed/30)}mo", row_limit
            else:
                # Untuk data sangat panjang
                return "max", row_limit

    def fetch_stock_data(self, ticker: str, interval: str, row_limit: int) -> pd.DataFrame:
        """
        Mengambil data historis saham untuk ticker yang diberikan.

        Args:
            ticker (str): Simbol saham (misalnya, "BBCA.JK")
            interval (str): Interval data (contoh: '15m')
            row_limit (int): Maksimal jumlah baris data yang diinginkan.

        Returns:
            pd.DataFrame: DataFrame yang berisi data saham yang diambil.
        """
        try:
            stock = yf.Ticker(ticker)

            # Dapatkan periode yang valid dan adjusted row limit
            period, adjusted_row_limit = self.calculate_period(row_limit, interval)
            logger.info(f"Mengambil data {ticker} dengan period='{period}' dan interval='{interval}'")

            data = stock.history(period=period, interval=interval)

            if data.empty:
                logger.error(f"Data retrieval unsuccessful for {ticker} with interval {interval}: Dataset kosong.")
                return pd.DataFrame()

            # Log informasi tentang jumlah baris yang berhasil diambil
            logger.info(f"Berhasil mengambil {len(data)} baris data untuk {ticker} ({interval})")

            # Jika data yang diambil lebih banyak dari adjusted_row_limit, ambil baris terakhir sesuai limit
            if len(data) > adjusted_row_limit:
                data = data.tail(adjusted_row_limit)

            # Tambahkan kolom Ticker untuk identifikasi
            data['Ticker'] = ticker

            # Reset index agar tanggal menjadi kolom reguler
            data = data.reset_index()

            return data

        except Exception as ex:
            logger.exception(f"Gagal mengambil data saham {ticker} ({interval}): {ex}")
            return pd.DataFrame()

    def save_data_to_csv(self, df: pd.DataFrame, ticker: str, interval: str) -> Optional[str]:
        """
        Menyimpan data ke file CSV.

        Args:
            df (pd.DataFrame): DataFrame yang akan disimpan
            ticker (str): Simbol saham
            interval (str): Interval data

        Returns:
            Optional[str]: Path ke file CSV jika berhasil, None jika gagal
        """
        if df.empty:
            logger.warning(f"Tidak ada data untuk disimpan: {ticker} ({interval})")
            return None

        try:
            # Format nama file: TICKER_INTERVAL.csv (contoh: BBCA_JK_1d.csv)
            ticker_clean = ticker.replace('.', '_')  # Ganti titik dengan underscore untuk nama file
            filename = f"{ticker_clean}_{interval}.csv"
            filepath = os.path.join(self.output_dir, filename)

            df.to_csv(filepath, index=False)
            logger.info(f"Data berhasil disimpan: {filepath} ({len(df)} baris)")
            return filepath
        except Exception as ex:
            logger.exception(f"Gagal menyimpan data ke file untuk {ticker} ({interval}): {ex}")
            return None

    def fetch_and_save_for_ticker(self, ticker: str, intervals: List[str], row_limit: int) -> Dict[str, Optional[str]]:
        """
        Mengambil dan menyimpan data untuk satu ticker dengan beberapa interval.

        Args:
            ticker (str): Simbol saham
            intervals (List[str]): List interval data yang akan diambil
            row_limit (int): Maksimal jumlah baris data

        Returns:
            Dict[str, Optional[str]]: Dictionary dengan interval sebagai key dan path file sebagai value
        """
        results = {}

        for interval in intervals:
            df = self.fetch_stock_data(ticker, interval, row_limit)
            file_path = self.save_data_to_csv(df, ticker, interval)
            results[interval] = file_path

        return results

    def fetch_multi_ticker_multi_timeframe(self,
                                          tickers: List[str],
                                          intervals: List[str],
                                          row_limit: int,
                                          use_threads: bool = True) -> Dict[str, Dict[str, Optional[str]]]:
        """
        Mengambil data untuk beberapa ticker dan beberapa interval.

        Args:
            tickers (List[str]): List simbol saham
            intervals (List[str]): List interval data
            row_limit (int): Maksimal jumlah baris data
            use_threads (bool): Gunakan multithreading untuk pengambilan data

        Returns:
            Dict[str, Dict[str, Optional[str]]]: Nested dictionary dengan struktur {ticker: {interval: filepath}}
        """
        logger.info(f"Memulai pengambilan data untuk {len(tickers)} ticker dengan {len(intervals)} timeframe")
        results = {}

        start_time = datetime.now()

        if use_threads:
            # Menggunakan ThreadPoolExecutor untuk menjalankan secara paralel
            with ThreadPoolExecutor(max_workers=min(10, len(tickers))) as executor:
                future_to_ticker = {
                    executor.submit(self.fetch_and_save_for_ticker, ticker, intervals, row_limit): ticker
                    for ticker in tickers
                }

                for future in future_to_ticker:
                    ticker = future_to_ticker[future]
                    try:
                        results[ticker] = future.result()
                    except Exception as ex:
                        logger.exception(f"Error dalam thread untuk ticker {ticker}: {ex}")
                        results[ticker] = {interval: None for interval in intervals}
        else:
            # Menggunakan pendekatan sekuensial
            for ticker in tickers:
                results[ticker] = self.fetch_and_save_for_ticker(ticker, intervals, row_limit)

        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds()

        logger.info(f"Pengambilan data selesai dalam {duration:.2f} detik")
        return results

    def generate_summary(self, results: Dict[str, Dict[str, Optional[str]]]) -> None:
        """
        Mencetak ringkasan hasil pengambilan data.

        Args:
            results (Dict[str, Dict[str, Optional[str]]]): Hasil pengambilan data
        """
        success_count = 0
        failed_count = 0

        print("\n" + "="*80)
        print("RINGKASAN PENGAMBILAN DATA SAHAM")
        print("="*80)

        for ticker, intervals in results.items():
            print(f"\nTicker: {ticker}")
            print("-" * 50)

            for interval, file_path in intervals.items():
                if file_path:
                    status = "BERHASIL"
                    success_count += 1
                else:
                    status = "GAGAL"
                    failed_count += 1

                print(f"  {interval}: {status} {file_path if file_path else ''}")

        print("\n" + "="*80)
        print(f"Total: {success_count + failed_count} pengambilan data")
        print(f"Berhasil: {success_count}, Gagal: {failed_count}")
        print("="*80 + "\n")

def main():
    # Daftar saham yang akan diambil datanya
    tickers = ["PGEO.JK", "ANTM.JK", "PTBA.JK", "FCX", "TINS.JK",
               "BBRI.JK", "BMRI.JK", "BBNI.JK", "INCO.JK", "TLKM.JK"]

    # Daftar interval/timeframe yang akan diambil
    intervals = ["15m", "30m", "1h", "1d"]

    # Jumlah baris data yang diinginkan
    row_limit = 1000

    # Direktori untuk menyimpan hasil
    output_dir = "stock_data"

    # Inisialisasi fetcher
    fetcher = StockDataFetcher(output_dir=output_dir)

    # Tampilkan perkiraan jumlah maksimum baris untuk setiap interval
    print("\nPERKIRAAN JUMLAH MAKSIMUM BARIS DATA PER INTERVAL:")
    print("-" * 50)
    for interval in intervals:
        max_rows = fetcher.get_max_rows_for_interval(interval)
        print(f"Interval {interval}: {max_rows} baris")

    # Ambil dan simpan data
    results = fetcher.fetch_multi_ticker_multi_timeframe(
        tickers=tickers,
        intervals=intervals,
        row_limit=row_limit,
        use_threads=True
    )

    # Tampilkan ringkasan hasil
    fetcher.generate_summary(results)

if __name__ == "__main__":
    main()

## Pre-Processing Data

In [ ]:
import pandas as pd
import os
import glob

# Direktori tempat file CSV disimpan
data_dir = "stock_data/"

# Mencari semua file CSV dalam direktori
csv_files = glob.glob(os.path.join(data_dir, "*.csv"))

# Nama kolom baru yang akan digunakan
new_columns = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits', 'Ticker']

# Loop melalui semua file CSV
for file_path in csv_files:
    try:
        # Membaca file CSV
        df = pd.read_csv(file_path)

        # Mengekstrak ticker dari nama file
        file_name = os.path.basename(file_path)
        ticker = file_name.split('_')[0]
        if 'JK' in file_name:
            ticker = ticker + '.JK'

        # Memeriksa jumlah kolom untuk memastikan data konsisten
        if len(df.columns) >= 6:  # Minimal memiliki OHLCV + minimal 1 kolom lain
            # Mengganti nama kolom
            df.columns = new_columns[:len(df.columns)]

            # Menambahkan kolom Ticker jika belum ada
            if 'Ticker' not in df.columns:
                df['Ticker'] = ticker

            # Mengkonversi tipe data dengan penanganan timezone
            # Menambahkan parameter utc=True untuk mengatasi warning
            df['Datetime'] = pd.to_datetime(df['Datetime'], utc=True)

            # Alternatif jika ingin zona waktu lokal:
            # df['Datetime'] = pd.to_datetime(df['Datetime']).dt.tz_localize(None)

            df['Open'] = df['Open'].astype(float)
            df['High'] = df['High'].astype(float)
            df['Low'] = df['Low'].astype(float)
            df['Close'] = df['Close'].astype(float)

            # Menghapus kolom Dividends dan Stock Splits jika ada
            columns_to_keep = [col for col in df.columns if col not in ['Dividends', 'Stock Splits']]
            df = df[columns_to_keep]

            # Menyimpan kembali file CSV
            df.to_csv(file_path, index=False)
            print(f"Preprocessing berhasil: {file_path}")
        else:
            print(f"Format tidak sesuai: {file_path} (jumlah kolom: {len(df.columns)})")
    except Exception as e:
        print(f"Error memproses {file_path}: {str(e)}")

print("\nPreprocessing selesai untuk semua file!")

## Push to Supabase

In [ ]:
import pandas as pd
import os
from supabase import create_client, Client
from datetime import datetime
import time
import glob
from google.colab import userdata

# Supabase credentials
supabase_url = userdata.get('SUPABASE_URL')
supabase_key = userdata.get('SUPABASE_KEY')

# Initialize Supabase client
supabase: Client = create_client(supabase_url, supabase_key)

print("Supabase client initialized successfully!")

# Define tickers and timeframes based on your data
tickers = [
    'PGEO.JK', 'ANTM.JK', 'PTBA.JK', 'FCX', 'TINS.JK',
    'BBRI.JK', 'BMRI.JK', 'BBNI.JK', 'INCO.JK', 'TLKM.JK'
]
timeframes = ['15m', '30m', '1h', '1d']

In [ ]:
def read_stock_csv(file_path):
    """Read a stock CSV file and return a pandas DataFrame."""
    try:
        df = pd.read_csv(file_path)
        if 'Datetime' in df.columns:
            df['Datetime'] = pd.to_datetime(df['Datetime'])
        return df
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None

def upload_dataframe_to_supabase(df, ticker, timeframe, table_name='stock_prices'):
    """Upload a DataFrame to Supabase table in chunks."""
    if df is None or df.empty:
        print(f"No data to upload for {ticker} - {timeframe}")
        return

    df['Ticker'] = ticker
    df['Timeframe'] = timeframe

    # ✅ Serialize datetime to ISO string
    if 'Datetime' in df.columns:
        df['Datetime'] = df['Datetime'].dt.strftime('%Y-%m-%dT%H:%M:%S')

    records = df.to_dict('records')
    chunk_size = 1000
    chunks = [records[i:i + chunk_size] for i in range(0, len(records), chunk_size)]

    total_uploaded = 0
    for i, chunk in enumerate(chunks):
        try:
            response = supabase.table(table_name).insert(chunk).execute()

            if hasattr(response, 'error') and response.error:
                print(f"Error uploading chunk {i+1} for {ticker} - {timeframe}: {response.error}")
            else:
                chunk_uploaded = len(chunk)
                total_uploaded += chunk_uploaded
                print(f"✅ Uploaded {chunk_uploaded} records for {ticker} - {timeframe} (chunk {i+1}/{len(chunks)})")

            time.sleep(0.1)

        except Exception as e:
            print(f"❌ Exception uploading chunk {i+1} for {ticker} - {timeframe}: {e}")

    print(f"✅ Total uploaded for {ticker} - {timeframe}: {total_uploaded}")
    return total_uploaded

def upload_all_stock_data_auto(base_dir='stock_data'):
    """Automatically find and upload all stock data CSV files."""
    total_records = 0
    csv_files = glob.glob(f"{base_dir}/*.csv")

    for file_path in csv_files:
        filename = os.path.basename(file_path)
        if '_' not in filename or not filename.endswith('.csv'):
            print(f"Skipping file: {filename}")
            continue

        parts = filename.replace('.csv', '').split('_')
        if len(parts) < 2:
            print(f"Skipping file with unexpected format: {filename}")
            continue

        timeframe = parts[-1]
        ticker_parts = parts[:-1]

        # Reconstruct ticker
        if len(ticker_parts) == 2 and ticker_parts[1] == 'JK':
            ticker = f"{ticker_parts[0]}.{ticker_parts[1]}"
        else:
            ticker = '_'.join(ticker_parts)

        print(f"\n🔄 Processing {ticker} - {timeframe} from {file_path}")

        df = read_stock_csv(file_path)
        records_uploaded = upload_dataframe_to_supabase(df, ticker, timeframe)
        if records_uploaded:
            total_records += records_uploaded

    print(f"\n✅ Total all records uploaded to Supabase: {total_records}")
    return total_records

# company_profitabilities

## Scraping

In [ ]:
import yfinance as yf
import pandas as pd
import logging
import os
from concurrent.futures import ThreadPoolExecutor
from typing import List, Dict

# Konfigurasi logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class CompanyInfoFetcher:
    """
    Fetcher untuk mengambil data return on equity dan return on assets dari Yahoo Finance.
    """

    def __init__(self, output_dir: str = "company_data"):
        self.output_dir = output_dir
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            logger.info(f"Output directory created: {output_dir}")

    def fetch_company_info(self, ticker: str) -> Dict:
        """
        Ambil data ticker, returnOnEquity, returnOnAssets.
        """
        try:
            logger.info(f"Fetching company profitability for {ticker}")
            stock = yf.Ticker(ticker)
            info = stock.info

            return {
                'ticker': ticker,
                'returnOnEquity': info.get('returnOnEquity', None),
                'returnOnAssets': info.get('returnOnAssets', None)
            }

        except Exception as ex:
            logger.exception(f"Failed to fetch data for {ticker}: {ex}")
            return {
                'ticker': ticker,
                'returnOnEquity': None,
                'returnOnAssets': None
            }

    def fetch_for_ticker(self, ticker: str) -> Dict:
        return self.fetch_company_info(ticker)

    def fetch_multi_tickers(self, tickers: List[str], use_threads: bool = True) -> pd.DataFrame:
        logger.info(f"Fetching data for {len(tickers)} tickers...")
        data_list = []

        if use_threads:
            with ThreadPoolExecutor(max_workers=min(10, len(tickers))) as executor:
                futures = {executor.submit(self.fetch_for_ticker, ticker): ticker for ticker in tickers}
                for future in futures:
                    try:
                        data = future.result()
                        data_list.append(data)
                    except Exception as ex:
                        logger.exception(f"Thread error: {ex}")
        else:
            for ticker in tickers:
                data = self.fetch_for_ticker(ticker)
                data_list.append(data)

        df = pd.DataFrame(data_list)
        return df

def main():
    tickers = [
        "PGEO.JK", "ANTM.JK", "PTBA.JK", "FCX", "TINS.JK",
        "BBRI.JK", "BMRI.JK", "BBNI.JK", "INCO.JK", "TLKM.JK"
    ]

    output_dir = "company_data"
    fetcher = CompanyInfoFetcher(output_dir=output_dir)

    df = fetcher.fetch_multi_tickers(tickers, use_threads=True)

    output_path = os.path.join(output_dir, "company_profitabilities.csv")
    df.to_csv(output_path, index=False)

    logger.info(f"✅ CSV saved at: {output_path}")
    print(f"\n🎉 DONE! File saved as: {output_path}")
    print(df.head())

if __name__ == "__main__":
    main()


🎉 DONE! File saved as: company_data/company_profitabilities.csv
    ticker  returnOnEquity  returnOnAssets
0  PGEO.JK         0.08055         0.04367
1  ANTM.JK             NaN             NaN
2  PTBA.JK         0.23252         0.08583
3      FCX         0.15686         0.08082
4  TINS.JK         0.17334         0.08412


## Push to Supabase

In [ ]:
import time
from supabase import create_client, Client
from google.colab import userdata

def upload_to_supabase():
    supabase_url = userdata.get('SUPABASE_URL')
    supabase_key = userdata.get('SUPABASE_KEY')
    supabase: Client = create_client(supabase_url, supabase_key)
    print("Supabase client initialized successfully!")

    df = pd.read_csv("/content/company_data/company_profitabilities.csv")
    df.columns = [col.lower() for col in df.columns]

    # Optional: clear existing data
    supabase.table("company_profitabilities").delete().neq('ticker', '').execute()
    print("🧹 Old data cleared")

    for _, row in df.iterrows():
        data = {
            "ticker": row['ticker'],
            "returnonequity": row['returnonequity'],
            "returnonassets": row['returnonassets']
        }
        supabase.table("company_profitabilities").insert(data).execute()
        time.sleep(0.1)

    print("🚀 All data uploaded to Supabase!")

if __name__ == "__main__":
    main()
    upload_to_supabase()

Supabase client initialized successfully!
🧹 Old data cleared
🚀 All data uploaded to Supabase!


# company_dividend

## Scraping

In [ ]:
import yfinance as yf
import pandas as pd
import logging
import os
from concurrent.futures import ThreadPoolExecutor
from typing import List, Dict

# Setup logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class CompanyDividendFetcher:
    """
    Fetcher untuk mengambil data dividend dari Yahoo Finance.
    """

    def __init__(self, output_dir: str = "company_data"):
        self.output_dir = output_dir
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            logger.info(f"📁 Output directory created: {output_dir}")

    def fetch_company_dividend(self, ticker: str) -> Dict:
        """
        Ambil data dividend info: dividendrate, dividendyield, exdividenddate, payoutratio.
        """
        try:
            logger.info(f"📥 Fetching dividend data for {ticker}")
            stock = yf.Ticker(ticker)
            info = stock.info

            return {
                'ticker': ticker,
                'dividendrate': info.get('dividendRate', None),
                'dividendyield': info.get('dividendYield', None),
                'exdividenddate': info.get('exDividendDate', None),
                'payoutratio': info.get('payoutRatio', None)
            }

        except Exception as ex:
            logger.exception(f"❌ Failed to fetch data for {ticker}: {ex}")
            return {
                'ticker': ticker,
                'dividendrate': None,
                'dividendyield': None,
                'exdividenddate': None,
                'payoutratio': None
            }

    def fetch_multi_tickers(self, tickers: List[str], use_threads: bool = True) -> pd.DataFrame:
        logger.info(f"🚀 Fetching dividend data for {len(tickers)} tickers...")
        data_list = []

        if use_threads:
            with ThreadPoolExecutor(max_workers=min(10, len(tickers))) as executor:
                futures = {executor.submit(self.fetch_company_dividend, ticker): ticker for ticker in tickers}
                for future in futures:
                    try:
                        data = future.result()
                        data_list.append(data)
                    except Exception as ex:
                        logger.exception(f"Thread error: {ex}")
        else:
            for ticker in tickers:
                data = self.fetch_company_dividend(ticker)
                data_list.append(data)

        df = pd.DataFrame(data_list)
        return df

def main():
    tickers = [
        "PGEO.JK", "ANTM.JK", "PTBA.JK", "FCX", "TINS.JK",
        "BBRI.JK", "BMRI.JK", "BBNI.JK", "INCO.JK", "TLKM.JK"
    ]

    output_dir = "company_data"
    fetcher = CompanyDividendFetcher(output_dir=output_dir)

    df = fetcher.fetch_multi_tickers(tickers, use_threads=True)

    # Format exdividenddate ke tanggal (kalau bentuknya epoch timestamp)
    if df["exdividenddate"].notnull().any():
        df["exdividenddate"] = pd.to_datetime(df["exdividenddate"], unit='s', errors='coerce').dt.date

    output_path = os.path.join(output_dir, "company_dividend.csv")
    df.to_csv(output_path, index=False)

    logger.info(f"✅ CSV saved at: {output_path}")
    print(f"\n🎉 DONE! File saved as: {output_path}")
    print(df.head())

if __name__ == "__main__":
    main()


🎉 DONE! File saved as: company_data/company_dividend.csv
    ticker  dividendrate  dividendyield exdividenddate  payoutratio
0  PGEO.JK         47.77           5.69     2024-06-06       0.7739
1  ANTM.JK        128.07           6.58     2024-05-21       1.2664
2  PTBA.JK        397.71          14.57     2024-05-21       0.8957
3      FCX          0.60           1.82     2025-04-15       0.4615
4  TINS.JK           NaN            NaN     2023-06-26       0.0000


## Push to Supabase

In [ ]:
import time
import pandas as pd
from supabase import create_client, Client
from google.colab import userdata
import math

def safe_float(val, max_abs_value=1e6):  # default max 1 juta, bisa override
    try:
        if pd.isna(val) or val in ['NaT', 'nan'] or (isinstance(val, float) and math.isinf(val)):
            return None
        val = float(val)
        if abs(val) > max_abs_value:
            return None
        return round(val, 6)  # cukup aman buat 99% kolom
    except:
        return None

def upload_to_supabase():
    supabase_url = userdata.get('SUPABASE_URL')
    supabase_key = userdata.get('SUPABASE_KEY')
    supabase: Client = create_client(supabase_url, supabase_key)
    print("✅ Supabase client initialized successfully!")

    df = pd.read_csv("/content/company_data/company_dividend.csv")
    df.columns = [col.lower() for col in df.columns]

    # Bersihin data sebelumnya
    supabase.table("company_dividend").delete().neq('ticker', '').execute()
    print("🧹 Old data in `company_dividend` cleared")

    for _, row in df.iterrows():
        data = {
            "ticker": row['ticker'],
            "dividendrate": safe_float(row['dividendrate'], max_abs_value=9.9999),
            "dividendyield": safe_float(row['dividendyield']),
            "exdividenddate": row['exdividenddate'],  # string, tetap kirim as is
            "payoutratio": safe_float(row['payoutratio']),
        }
        supabase.table("company_dividend").insert(data).execute()
        time.sleep(0.1)

    print("🚀 Dividend data uploaded to Supabase!")

if __name__ == "__main__":
    main()  # make sure this generates the CSV before upload
    upload_to_supabase()


# company_growth

## Scraping

In [ ]:
import yfinance as yf
import pandas as pd
import logging
import os
from concurrent.futures import ThreadPoolExecutor
from typing import List, Dict

# Konfigurasi logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class CompanyInfoFetcher:
    """
    Fetcher untuk mengambil data pertumbuhan perusahaan dari Yahoo Finance.
    """

    def __init__(self, output_dir: str = "company_data"):
        self.output_dir = output_dir
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            logger.info(f"Output directory created: {output_dir}")

    def fetch_company_growth(self, ticker: str) -> Dict:
        """
        Ambil data ticker, revenueGrowth, earningsGrowth, earningsQuarterlyGrowth.
        """
        try:
            logger.info(f"Fetching company growth for {ticker}")
            stock = yf.Ticker(ticker)
            info = stock.info

            return {
                'ticker': ticker,
                'revenuegrowth': info.get('revenueGrowth', None),
                'earningsgrowth': info.get('earningsGrowth', None),
                'earningsquarterlygrowth': info.get('earningsQuarterlyGrowth', None)
            }

        except Exception as ex:
            logger.exception(f"Failed to fetch data for {ticker}: {ex}")
            return {
                'ticker': ticker,
                'revenuegrowth': None,
                'earningsgrowth': None,
                'earningsquarterlygrowth': None
            }

    def fetch_for_ticker(self, ticker: str) -> Dict:
        return self.fetch_company_growth(ticker)

    def fetch_multi_tickers(self, tickers: List[str], use_threads: bool = True) -> pd.DataFrame:
        logger.info(f"Fetching data for {len(tickers)} tickers...")
        data_list = []

        if use_threads:
            with ThreadPoolExecutor(max_workers=min(10, len(tickers))) as executor:
                futures = {executor.submit(self.fetch_for_ticker, ticker): ticker for ticker in tickers}
                for future in futures:
                    try:
                        data = future.result()
                        data_list.append(data)
                    except Exception as ex:
                        logger.exception(f"Thread error: {ex}")
        else:
            for ticker in tickers:
                data = self.fetch_for_ticker(ticker)
                data_list.append(data)

        df = pd.DataFrame(data_list)
        return df

def main():
    tickers = [
        "PGEO.JK", "ANTM.JK", "PTBA.JK", "FCX", "TINS.JK",
        "BBRI.JK", "BMRI.JK", "BBNI.JK", "INCO.JK", "TLKM.JK"
    ]

    output_dir = "company_data"
    fetcher = CompanyInfoFetcher(output_dir=output_dir)

    df = fetcher.fetch_multi_tickers(tickers, use_threads=True)

    output_path = os.path.join(output_dir, "company_growth.csv")
    df.to_csv(output_path, index=False)

    logger.info(f"✅ CSV saved at: {output_path}")
    print(f"\n🎉 DONE! File saved as: {output_path}")
    print(df.head())

if __name__ == "__main__":
    main()


🎉 DONE! File saved as: company_data/company_growth.csv
    ticker  revenuegrowth  earningsgrowth  earningsquarterlygrowth
0  PGEO.JK          0.031          -0.143                   -0.119
1  ANTM.JK          1.561           5.313                    5.313
2  PTBA.JK          0.126          -0.198                   -0.195
3      FCX         -0.031          -0.292                   -0.294
4  TINS.JK          0.291             NaN                      NaN


## Push to Supabase

In [ ]:
import time
import pandas as pd
from supabase import create_client, Client
from google.colab import userdata
import math

def upload_to_supabase():
    supabase_url = userdata.get('SUPABASE_URL')
    supabase_key = userdata.get('SUPABASE_KEY')
    supabase: Client = create_client(supabase_url, supabase_key)
    print("✅ Supabase client initialized successfully!")

    df = pd.read_csv("/content/company_data/company_growth.csv")
    df.columns = [col.lower() for col in df.columns]

    # Optional: clear existing data
    supabase.table("company_growth").delete().neq('ticker', '').execute()
    print("🧹 Old data cleared")

    # NaN dan float yang gak valid akan diubah jadi None
    def safe_float(val):
        if pd.isna(val) or math.isinf(val):
            return None
        return float(val)

    for _, row in df.iterrows():
        data = {
            "ticker": row['ticker'],
            "revenuegrowth": safe_float(row['revenuegrowth']),
            "earningsgrowth": safe_float(row['earningsgrowth']),
            "earningsquarterlygrowth": safe_float(row['earningsquarterlygrowth'])
        }
        supabase.table("company_growth").insert(data).execute()
        time.sleep(0.1)

    print("🚀 All data uploaded to Supabase!")

if __name__ == "__main__":
    main()
    upload_to_supabase()


🎉 DONE! File saved as: company_data/company_growth.csv
    ticker  revenuegrowth  earningsgrowth  earningsquarterlygrowth
0  PGEO.JK          0.031          -0.143                   -0.119
1  ANTM.JK          1.561           5.313                    5.313
2  PTBA.JK          0.126          -0.198                   -0.195
3      FCX         -0.031          -0.292                   -0.294
4  TINS.JK          0.291             NaN                      NaN
✅ Supabase client initialized successfully!
🧹 Old data cleared
🚀 All data uploaded to Supabase!


# company_finance

## Scraping Data

In [ ]:
import yfinance as yf
import pandas as pd
import logging
import os
import json
from concurrent.futures import ThreadPoolExecutor
from typing import List, Dict, Optional

# Setup logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


class CompanyInfoFetcher:
    def __init__(self, output_dir: str = "company_data"):
        self.output_dir = output_dir
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            logger.info(f"📁 Output directory created: {output_dir}")

    def fetch_company_finance(self, ticker: str) -> Dict:
        try:
            logger.info(f"📊 Fetching financial info for {ticker}")
            stock = yf.Ticker(ticker)
            info = stock.info

            financial_data = {
                'ticker': ticker.upper(),  # Uppercase ticker
                'marketcap': info.get('marketCap'),
                'shareoutstanding': info.get('sharesOutstanding'),
                'totalrevenue': info.get('totalRevenue'),
                'netincometocommon': info.get('netIncomeToCommon'),
                'profitmargins': info.get('profitMargins'),
                'trailingeps': info.get('trailingEps'),
                'forwardeps': info.get('forwardEps'),
                'grossmargins': info.get('grossMargins'),
                'operatingmargins': info.get('operatingMargins'),
                'operatingcashflow': info.get('operatingCashflow'),
                'freecashflow': info.get('freeCashflow')
            }

            logger.info(f"✅ Success for {ticker}")
            return financial_data

        except Exception as ex:
            logger.exception(f"❌ Failed to fetch for {ticker}: {ex}")
            return {
                'ticker': ticker.upper(),
                'error': str(ex)
            }

    def fetch_multi_tickers(self, tickers: List[str], use_threads: bool = True) -> List[Dict]:
        logger.info(f"🚀 Starting to fetch {len(tickers)} tickers")
        results = []

        if use_threads:
            with ThreadPoolExecutor(max_workers=min(10, len(tickers))) as executor:
                futures = {
                    executor.submit(self.fetch_company_finance, ticker): ticker
                    for ticker in tickers
                }
                for future in futures:
                    ticker = futures[future]
                    try:
                        result = future.result()
                        results.append(result)
                    except Exception as ex:
                        logger.exception(f"Thread error for {ticker}: {ex}")
                        results.append({
                            'ticker': ticker.upper(),
                            'error': str(ex)
                        })
        else:
            for ticker in tickers:
                results.append(self.fetch_company_finance(ticker))

        logger.info("✅ All data fetched")
        return results

    def save_all_to_csv(self, data: List[Dict]) -> Optional[str]:
        if not data:
            logger.warning("⚠️ No data to save.")
            return None

        try:
            df = pd.DataFrame(data)
            filepath = os.path.join(self.output_dir, "all_financial_data.csv")
            df.to_csv(filepath, index=False)

            json_path = os.path.join(self.output_dir, "all_financial_data.json")
            with open(json_path, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=4)

            logger.info(f"💾 Saved CSV to {filepath}")
            logger.info(f"💾 Saved JSON to {json_path}")
            return filepath
        except Exception as ex:
            logger.exception(f"❌ Failed to save data: {ex}")
            return None

    def generate_summary(self, results: List[Dict]) -> None:
        success_count = sum(1 for r in results if 'error' not in r)
        failed_count = len(results) - success_count

        print("\n" + "=" * 80)
        print("📋 FINANCIAL DATA FETCHING SUMMARY")
        print("=" * 80)

        for result in results:
            status = "✅ SUCCESS" if 'error' not in result else "❌ FAILED"
            print(f"{result['ticker']}: {status}")

        print("\n" + "=" * 80)
        print(f"Total tickers: {len(results)}")
        print(f"✅ Success: {success_count}")
        print(f"❌ Failed: {failed_count}")
        print("=" * 80 + "\n")


def main():
    tickers = [
        "PGEO.JK", "ANTM.JK", "PTBA.JK", "FCX", "TINS.JK",
        "BBRI.JK", "BMRI.JK", "BBNI.JK", "INCO.JK", "TLKM.JK"
    ]
    output_dir = "company_data"

    fetcher = CompanyInfoFetcher(output_dir=output_dir)
    results = fetcher.fetch_multi_tickers(tickers, use_threads=True)
    fetcher.save_all_to_csv(results)
    fetcher.generate_summary(results)


if __name__ == "__main__":
    main()



📋 FINANCIAL DATA FETCHING SUMMARY
PGEO.JK: ✅ SUCCESS
ANTM.JK: ✅ SUCCESS
PTBA.JK: ✅ SUCCESS
FCX: ✅ SUCCESS
TINS.JK: ✅ SUCCESS
BBRI.JK: ✅ SUCCESS
BMRI.JK: ✅ SUCCESS
BBNI.JK: ✅ SUCCESS
INCO.JK: ✅ SUCCESS
TLKM.JK: ✅ SUCCESS

Total tickers: 10
✅ Success: 10
❌ Failed: 0



## Push to Supabase

In [ ]:
# upload_to_supabase.py

import pandas as pd
import time
import math
from supabase import create_client, Client
from google.colab import userdata

def upload_to_supabase():
    supabase_url = userdata.get('SUPABASE_URL')
    supabase_key = userdata.get('SUPABASE_KEY')
    supabase: Client = create_client(supabase_url, supabase_key)
    print("✅ Supabase client initialized successfully!")

    df = pd.read_csv("company_data/all_financial_data.csv")
    df.columns = [col.lower() for col in df.columns]

    # Optional: clear old data
    supabase.table("company_finance").delete().neq('ticker', '').execute()
    print("🧹 Old data cleared")

    def safe_float(val):
        if pd.isna(val) or math.isinf(val):
            return None
        return float(val)

    for _, row in df.iterrows():
        data = {
            "ticker": row.get('ticker'),
            "marketcap": safe_float(row.get('marketcap')),
            "shareoutstanding": safe_float(row.get('shareoutstanding')),
            "totalrevenue": safe_float(row.get('totalrevenue')),
            "netincometocommon": safe_float(row.get('netincometocommon')),
            "profitmargins": safe_float(row.get('profitmargins')),
            "trailingeps": safe_float(row.get('trailingeps')),
            "forwardeps": safe_float(row.get('forwardeps')),
            "grossmargins": safe_float(row.get('grossmargins')),
            "operatingmargins": safe_float(row.get('operatingmargins')),
            "operatingcashflow": safe_float(row.get('operatingcashflow')),
            "freecashflow": safe_float(row.get('freecashflow'))
        }
        supabase.table("company_finance").insert(data).execute()
        time.sleep(0.1)

    print("🚀 Data uploaded to Supabase!")

if __name__ == "__main__":
    upload_to_supabase()


✅ Supabase client initialized successfully!
🧹 Old data cleared
🚀 Data uploaded to Supabase!


# company_valuation

## Scraping Data

In [ ]:
import yfinance as yf
import pandas as pd
import logging
import os
import json
from concurrent.futures import ThreadPoolExecutor
from typing import List, Dict, Optional

# Logging setup
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class CompanyValuationFetcher:
    def __init__(self, output_dir: str = "company_data"):
        self.output_dir = output_dir
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            logger.info(f"Output directory created: {output_dir}")

    def fetch_company_valuation(self, ticker: str) -> Dict:
        try:
            logger.info(f"Fetching valuation info for {ticker}")
            stock = yf.Ticker(ticker)
            info = stock.info

            valuation_data = {
                'ticker': ticker.upper(),
                'trailingpe': info.get('trailingPE'),
                'forwardpe': info.get('forwardPE'),
                'pegratio': info.get('pegRatio'),
                'pricetobook': info.get('priceToBook'),
                'pricetosalestrailing12months': info.get('priceToSalesTrailing12Months')
            }

            logger.info(f"Successfully fetched valuation info for {ticker}")
            return valuation_data

        except Exception as ex:
            logger.exception(f"Failed to fetch valuation info for {ticker}: {ex}")
            return {
                'ticker': ticker.upper(),
                'error': str(ex)
            }

    def fetch_multi_tickers(self, tickers: List[str], use_threads: bool = True) -> List[Dict]:
        logger.info(f"Starting data fetching for {len(tickers)} tickers")
        results = []

        if use_threads:
            with ThreadPoolExecutor(max_workers=min(10, len(tickers))) as executor:
                futures = {
                    executor.submit(self.fetch_company_valuation, ticker): ticker
                    for ticker in tickers
                }
                for future in futures:
                    ticker = futures[future]
                    try:
                        result = future.result()
                        results.append(result)
                    except Exception as ex:
                        logger.exception(f"Error in thread for ticker {ticker}: {ex}")
                        results.append({
                            'ticker': ticker.upper(),
                            'error': str(ex)
                        })
        else:
            for ticker in tickers:
                results.append(self.fetch_company_valuation(ticker))

        logger.info("Data fetching completed")
        return results

    def save_all_to_csv(self, data: List[Dict]) -> Optional[str]:
        if not data:
            logger.warning("No data to save.")
            return None

        try:
            df = pd.DataFrame(data)
            filepath = os.path.join(self.output_dir, "company_valuation_data.csv")
            df.to_csv(filepath, index=False)

            json_path = os.path.join(self.output_dir, "company_valuation_data.json")
            with open(json_path, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=4)

            logger.info(f"All valuation data saved to {filepath}")
            return filepath
        except Exception as ex:
            logger.exception(f"Failed to save data: {ex}")
            return None

    def generate_summary(self, results: List[Dict]) -> None:
        success_count = sum(1 for r in results if 'error' not in r)
        failed_count = len(results) - success_count

        print("\n" + "=" * 80)
        print("VALUATION DATA FETCHING SUMMARY")
        print("=" * 80)

        for result in results:
            status = "SUCCESS" if 'error' not in result else "FAILED"
            print(f"{result['ticker']}: {status}")

        print("\n" + "=" * 80)
        print(f"Total: {len(results)} tickers")
        print(f"Success: {success_count}, Failed: {failed_count}")
        print("=" * 80 + "\n")


def main():
    tickers = [
        "PGEO.JK", "ANTM.JK", "PTBA.JK", "FCX", "TINS.JK",
        "BBRI.JK", "BMRI.JK", "BBNI.JK", "INCO.JK", "TLKM.JK"
    ]
    output_dir = "company_data"

    fetcher = CompanyValuationFetcher(output_dir=output_dir)
    results = fetcher.fetch_multi_tickers(tickers, use_threads=True)
    fetcher.save_all_to_csv(results)
    fetcher.generate_summary(results)


if __name__ == "__main__":
    main()



VALUATION DATA FETCHING SUMMARY
PGEO.JK: SUCCESS
ANTM.JK: SUCCESS
PTBA.JK: SUCCESS
FCX: SUCCESS
TINS.JK: SUCCESS
BBRI.JK: SUCCESS
BMRI.JK: SUCCESS
BBNI.JK: SUCCESS
INCO.JK: SUCCESS
TLKM.JK: SUCCESS

Total: 10 tickers
Success: 10, Failed: 0



## Push to Supabase

In [ ]:
# upload_to_supabase.py

import pandas as pd
import time
import math
from supabase import create_client, Client
from google.colab import userdata

def upload_to_supabase():
    supabase_url = userdata.get('SUPABASE_URL')
    supabase_key = userdata.get('SUPABASE_KEY')
    supabase: Client = create_client(supabase_url, supabase_key)
    print("✅ Supabase client initialized successfully!")

    df = pd.read_csv("company_data/company_valuation_data.csv")
    df.columns = [col.lower() for col in df.columns]

    # Optional: clear old data
    supabase.table("company_valuation").delete().neq('ticker', '').execute()
    print("🧹 Old data cleared")

    def safe_float(val):
        if pd.isna(val) or math.isinf(val):
            return None
        return float(val)

    for _, row in df.iterrows():
        data = {
            "ticker": row.get('ticker'),
            "trailingpe": safe_float(row.get('trailingpe')),
            "forwardpe": safe_float(row.get('forwardpe')),
            "pegratio": safe_float(row.get('pegratio')),
            "pricetobook": safe_float(row.get('pricetobook')),
            "pricetosalestrailing12months": safe_float(row.get('pricetosalestrailing12months'))
        }
        supabase.table("company_valuation").insert(data).execute()
        time.sleep(0.1)

    print("🚀 Data uploaded to Supabase!")

if __name__ == "__main__":
    upload_to_supabase()


✅ Supabase client initialized successfully!
🧹 Old data cleared
🚀 Data uploaded to Supabase!
